In [1]:
#!/usr/bin/env python
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt

In [2]:
import numpy as np, os, sys
from tqdm.notebook import tqdm    
from manipulations import get_classes, get_classes_from_header, get_Fs_from_header, load_challenge_data
from sklearn.model_selection import KFold
if __name__ == '__main__':
    
    Datas = []
    Header_datas = []
    Classes = []
    dataset_idx = {}
    dataset_train_idx = {}
    dataset_test_idx = {}
    global_idx = 0
    for dataset in [5]:
        print('Dataset ', dataset)
        # Parse arguments.
        if len(sys.argv) != 3:
            raise Exception('Include the input and output directories as arguments, e.g., python driver.py input output.')

        input_directory = '../Data/Training{}/'.format(dataset)
        output_directory = '../Output/'

        # Find files.
        input_files = []
        for f in os.listdir(input_directory):
            if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
                input_files.append(f)

        if not os.path.isdir(output_directory):
            os.mkdir(output_directory)

        classes=get_classes(input_directory,input_files)

        num_files = len(input_files)
        datas = []
        header_datas = []
        dataset_idx[dataset] = []
        for i, f in tqdm(enumerate(input_files)):
            #print('    {}/{}...'.format(i+1, num_files), f)
            tmp_input_file = os.path.join(input_directory,f)
            data,header_data = load_challenge_data(tmp_input_file)
            datas.append(data[:,1000:7000])
            header_datas.append(header_data)
            dataset_idx[dataset].append(global_idx)
            global_idx += 1

        Datas += datas
        Header_datas += header_datas
        Classes += classes
        
        
        kf = KFold(5)
        train_idx, test_idx = next(kf.split(dataset_idx[dataset]))
        dataset_train_idx[dataset] = train_idx
        dataset_test_idx[dataset] = test_idx
        print('Done.')

Dataset  5



Done.


In [3]:
all_train_idx = []
for dataset in [5]:
    all_train_idx.extend(dataset_train_idx[dataset])
    
all_test_idx = []
for dataset in [5]:
    all_test_idx.extend(dataset_test_idx[dataset])

In [4]:
from manipulations import get_abbr, get_name
from global_vars import labels, Dx_map, Dx_map_unscored
first_idx = {scored_code: None for scored_code in list(Dx_map['SNOMED CT Code'])}
first_idx_unscored = {unscored_code: None for unscored_code in list(Dx_map_unscored['SNOMED CT Code'])}

# for i, Header_data in tqdm(enumerate(Header_datas)):
#     codes = get_classes_from_header(Header_data)
#     abbrs = ' '.join([get_abbr(int(code), Dx_map, Dx_map_unscored) for code in codes])
#     for code, abbr in zip(codes, abbrs):
#         if code in first_idx and first_idx[code] is None:
#             first_idx[code] = i
#         if code in first_idx_unscored and first_idx_unscored[code] is None:
#             first_idx_unscored[code] = i
#     encore = False
#     for code in first_idx.keys():
#         if first_idx[code] is None:
#             encore = True
#             break
# #     for code in first_idx_unscored.keys():
# #         if first_idx_unscored[code] is None:
# #             encore = True
# #             break
#     if not encore:
#         break

In [5]:
# from signal_processing import myfilter, main_QRST
# Codes = []
# Q_locs = []
# idxes = []
# for idx in tqdm(range(0, len(Datas))): 
#     if np.sum(Datas[idx]) != 0: # not all zeros
#         codes = get_classes_from_header(Header_datas[idx])
#         names = ', '.join([get_name(int(code), Dx_map, Dx_map_unscored) for code in codes])

#         filtered_Data = myfilter(Datas[idx], 500, vis=False)

#         # get the lead to apply Pan Tomkins
#         Q_loc = main_QRST(filtered_Data, idx, '', '', names, fig2=False)

#         # store
#         Codes.append(codes)
#         Q_locs.append(Q_loc)
#     else:
#         Codes.append([])
#         Q_locs.append([])
#     idxes.append(idx)


In [6]:
# import pickle
# with open('../saved/idxes_1000_7000.pkl', 'wb') as idxes_file:
#     pickle.dump(idxes, idxes_file)

# with open('../saved/Codes_1000_7000.pkl', 'wb') as Codes_file:
#     pickle.dump(Codes, Codes_file)

# with open('../saved/Q_locs_1000_7000.pkl', 'wb') as Q_locs_file:
#     pickle.dump(Q_locs, Q_locs_file)


In [7]:
import pickle
Q_locs = None
with open('../saved/Q_locs_1000_7000.pkl', 'rb') as Q_locs_file:
    Q_locs = pickle.load(Q_locs_file)

Codes = None
with open('../saved/Codes_1000_7000.pkl', 'rb') as Codes_file:
    Codes = pickle.load(Codes_file)

idxes = None
with open('../saved/idxes_1000_7000.pkl', 'rb') as idxes_file:
    idxes = pickle.load(idxes_file)
    
Codes = []
for idx in tqdm(range(0, len(Datas))): 
    if np.sum(Datas[idx]) != 0: # not all zeros
        codes = get_classes_from_header(Header_datas[idx])
        # store
        Codes.append(codes)
    else:
        Codes.append([])

In [8]:
from global_vars import labels
def get_scored_class(code, labels):
    return [1 if label in code else 0 for label in labels]

In [9]:
data_labels = np.array([get_scored_class(codes, labels) for codes in Codes])
    

In [10]:
MAX_RR = 600 # 60 beats/min => 60 beats/60 s ==> beat/1s ==> 500 samples / beat

In [11]:
X_train = []
Data_labels_train = []
Idxes_train = []
Idxes_dict_train = {}
ct = 0
for i in tqdm(dataset_train_idx[5]):
    Q_loc = Q_locs[i]
    RR_avg = np.median([Q_loc[k+1] - Q_loc[k] for k in range(len(Q_loc)-1)])
    RR_th = (0.3 * RR_avg, 3 * RR_avg)
    
    ks = [k for k in range(len(Q_loc)-1) if Q_loc[k+1] - Q_loc[k] > RR_th[0] 
              and Q_loc[k+1] - Q_loc[k] < RR_th[1]]
    
    
    Idxes_dict_train[i] = []
    for k in ks:
        Data_labels_train.append(get_scored_class(Codes[i], labels))
        X_train.append(Datas[i][:,1000+Q_loc[k]:1000+Q_loc[k+1]])
        Idxes_train.append(i)
        Idxes_dict_train[i].append(ct)
        
    ct += 1

/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
X_test = []
Data_labels_test = []
Idxes_test = []
Idxes_dict_test = {}
ct = 0
for i in tqdm(dataset_test_idx[5]):
    Q_loc = Q_locs[i]
    RR_avg = np.median([Q_loc[k+1] - Q_loc[k] for k in range(len(Q_loc)-1)])
    RR_th = (0.3 * RR_avg, 3 * RR_avg)
    
    ks = [k for k in range(len(Q_loc)-1) if Q_loc[k+1] - Q_loc[k] > RR_th[0] 
              and Q_loc[k+1] - Q_loc[k] < RR_th[1]]
    
    
    Idxes_dict_test[i] = []
    for k in ks:
        Data_labels_test.append(get_scored_class(Codes[i], labels))
        X_test.append(Datas[i][:,1000+Q_loc[k]:1000+Q_loc[k+1]])
        Idxes_test.append(i)
        Idxes_dict_test[i].append(ct)
        
    ct += 1

In [13]:
Signals_train = np.zeros((len(X_train),12,MAX_RR))
for i in range(len(X_train)):
    Signals_train[i,:,:min(len(X_train[i][0]),MAX_RR)] = X_train[i][:,:MAX_RR]

In [14]:
Signals_test = np.zeros((len(X_test),12,MAX_RR))
for i in range(len(X_test)):
    Signals_test[i,:,:min(len(X_test[i][0]),MAX_RR)] = X_test[i][:,:MAX_RR]

# On y va

In [15]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/resnet1d_dataset5_cuda0')


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [17]:
from resnet1d import resnet18, ECGResNet
import torch
from torch import nn

In [18]:

def weighted_binary_cross_entropy2(sigmoid_x, y, weighted_matrix, weight=None, reduction=None):
    """
    Aha this is correct!
    sigmoid_x = nn.Sigmoid()(x)
    Args:
        sigmoid_x: predicted probability of size [N,C], N sample and C Class. Eg. Must be in range of [0,1]
        targets: true value, one-hot-like vector of size [N,C]
        pos_weight: Weight for postive sample
    """
    if not (y.size() == sigmoid_x.size()):
        raise ValueError("Target size ({}) must be the same as input size ({})".format(y.size(), sigmoid_x.size()))
   
    #print("y.size(), sigmoid_x.size()", y.size(), sigmoid_x.size())
    sigmoid_x = torch.clamp(sigmoid_x,min=1e-7,max=1-1e-7) 
    loss = - torch.matmul(y*sigmoid_x.log() + (1-y)*(1-sigmoid_x).log(), weighted_matrix)
    
    if weight is not None:
        loss = loss * weight
        
    if reduction is None:
        return loss
    elif reduction == 'mean':
        return loss.mean()
    elif reduction == 'sum':
        return loss.sum()
    return None
    
class WeightedBCELoss(nn.Module):
    def __init__(self, weights, PosWeightIsDynamic= False, WeightIsDynamic= False, 
                 reduction='mean'):
        """
        Args:
            pos_weight = Weight for postive samples. Size [1,C]
            weight = Weight for Each class. Size [1,C]
            PosWeightIsDynamic: If True, the pos_weight is computed on each batch. If pos_weight is None, then it remains None.
            WeightIsDynamic: If True, the weight is computed on each batch. If weight is None, then it remains None.
        """
        super().__init__()

        self.register_buffer('weights', weights)
        self.reduction = reduction
        self.PosWeightIsDynamic = PosWeightIsDynamic

    def forward(self, input, target):
        if self.PosWeightIsDynamic:
            positive_counts = target.sum(dim=0)
            nBatch = len(target)
            self.pos_weight = (nBatch - positive_counts)/(positive_counts +1e-5)


        return weighted_binary_cross_entropy2(input, target,
                                             weighted_matrix=self.weights,
                                             reduction=self.reduction)

In [19]:
Signal_12leads_train = np.transpose(Signals_train, (1,0,2))
Signal_12leads_test= np.transpose(Signals_test, (1,0,2))

In [20]:
from torch.utils.data import Dataset
from torchvision import transforms
from global_vars import labels
import os
class SignalDataset(Dataset):

    def __init__(self, signals, labels):
        self.signals = signals
        self.labels = labels

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample =(torch.cat([torch.Tensor(np.array([self.signals[channel,idx]]).transpose()) for channel in range(12)], axis=1), 
                  torch.Tensor(self.labels[idx]))

        return sample
    
signal_datasets_train = SignalDataset(Signal_12leads_train, Data_labels_train)
signal_datasets_test = SignalDataset(Signal_12leads_test, Data_labels_test)

In [21]:
Signal_12leads_train.shape

(12, 169804, 600)

In [22]:
signal_datasets_train[0][0].shape

torch.Size([600, 12])

In [23]:
from global_vars import weights

In [24]:
weights_tensor = torch.Tensor(weights).to(device)

In [25]:
from evaluation.evaluate_12ECG_score import compute_modified_confusion_matrix, compute_challenge_metric

In [26]:
normal_idx = np.argwhere(labels.to_numpy()==426783006)

In [27]:
def compute_score(labels, outputs, weights, normal_index=normal_idx):
    num_recordings, num_classes = np.shape(labels)
    # Compute the observed score.
    A = compute_modified_confusion_matrix(labels, outputs)
    observed_score = np.nansum(weights * A)

    # Compute the score for the model that always chooses the correct label(s).
    correct_outputs = labels
    A = compute_modified_confusion_matrix(labels, correct_outputs)
    correct_score = np.nansum(weights * A)

    # Compute the score for the model that always chooses the normal class.
    inactive_outputs = np.zeros((num_recordings, num_classes), dtype=np.bool)
    inactive_outputs[:, normal_index] = 1
    A = compute_modified_confusion_matrix(labels, inactive_outputs)
    inactive_score = np.nansum(weights * A)

    if correct_score != inactive_score:
        normalized_score = float(observed_score - inactive_score) / float(correct_score - inactive_score)
    else:
        normalized_score = float('nan')

    return normalized_score

In [28]:
pos_weight = np.ones(27) * 2
pos_weight = torch.Tensor(pos_weight).to(device)

In [ ]:
from torchvision import datasets, models, transforms
from myeval import agg_y_preds_bags, binary_acc, geometry_loss
import torch.optim as optim
from torch.optim import lr_scheduler
from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import GroupKFold
import time

st = time.time()
patience = 50
batch_size=65000

saved_dir = '../saved/ResNet1d/'



model = ECGResNet(MAX_RR, len(labels))
model.to(device)

trainDataset = torch.utils.data.Subset(signal_datasets_train, range(0,len(Signals_train)))
testDataset = torch.utils.data.Subset(signal_datasets_test, range(0,len(Signals_test)))

trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size, shuffle = True, pin_memory=True)#sampler = sampler)
trainLoader2 = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size, shuffle = False, pin_memory=True)#sampler = sampler)
testLoader = torch.utils.data.DataLoader(testDataset, batch_size = batch_size, shuffle = False, pin_memory=True)


optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) #
# Decay LR by a factor of 0.1 every 100 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.1)

#     criterion_train = WeightedBCELoss(weights=weights_tensor, reduction='mean')
#     criterion_test = WeightedBCELoss(weights=weights_tensor, reduction='mean')

criterion_train = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction='mean')
criterion_test = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction='mean')

losses_train = []
losses_test = []

avg_losses_train = []
avg_losses_test = []


early_stopping = EarlyStopping(patience, verbose=False, 
                              saved_dir=saved_dir, 
                               save_name='Resnet1'+str(i))
epoch = 0
auroc = 0
auprc = 0
accuracy = 0
fmeasure = 0
fbeta = 0
gbeta = 0
for epoch in range(50000):

    model.train()
    output_trains = []

    for X_train, y_train in trainLoader:
        y_train = y_train.to(device)
        X_train = X_train.to(device)
        optimizer.zero_grad()
        output_train = model(X_train)
        loss_train = criterion_train(output_train, y_train)
        #print(loss_train)
        losses_train.append(loss_train.item())
        loss_train.backward()
        optimizer.step()

    scheduler.step()

    avg_loss_train = np.average(losses_train)
    avg_losses_train.append(avg_loss_train)

    output_tests = []
    y_tests = []
    y_trains = []
    with torch.no_grad():
        model.eval()

        running_loss_train = 0.0
        for k, (X_train, y_train) in enumerate(trainLoader2):  
            X_train = X_train.to(device)
            output_train = model(X_train)
            output_trains.append(output_train.cpu())
            y_trains.append(y_train.cpu())

            avg_loss_train = np.average(losses_train)
            avg_losses_train.append(avg_loss_train)

            running_loss_train += loss_train.item() 
            writer.add_scalar('training loss',
                running_loss_train/(epoch * (len(train_idx)//batch_size+1) + k+1),
                epoch * (len(train_idx)//batch_size+1) + k)


        running_loss_test = 0.0
        for X_test, y_test in testLoader:  
            y_test = y_test.to(device)
            X_test = X_test.to(device)
            output_test = model(X_test)


            loss_test = criterion_test(output_test, y_test)
            losses_test.append(loss_test.item())

            running_loss_test += loss_test.item()

            output_tests.append(output_test.cpu())
            y_tests.append(y_test.cpu())

        avg_loss_test = np.average(losses_test)
        avg_losses_test.append(avg_loss_test)


        writer.add_scalar('testing loss',
                running_loss_test/(epoch* (len(train_idx)//(batch_size+1))+1),
                epoch* (len(train_idx)//batch_size+1))



    y_trains = torch.cat(y_trains, axis=0)
    y_tests = torch.cat(y_tests, axis=0)

    output_trains = torch.cat(output_trains, axis=0)
    y_train_preds = torch.sigmoid(output_trains)

    output_tests = torch.cat(output_tests, axis=0)
    y_test_preds = torch.sigmoid(output_tests)

    #output_trains = torch.cat(output_trains, axis=0)
#         y_train_preds_max, y_train_preds_mean, _ = agg_y_preds_bags(y_train_preds, bag_size=n_segments)
#         y_test_preds_max, y_test_preds_mean, _ = agg_y_preds_bags(y_test_preds, bag_size=n_segments)
#         _, _, y_trains = agg_y_preds_bags(y_trains, bag_size=n_segments)
#         _, _, y_tests = agg_y_preds_bags(y_tests, bag_size=n_segments)






    acc, fmeasure, fbeta, gbeta = binary_acc(y_train_preds, y_trains)           
    acc2, fmeasure2, fbeta2, gbeta2 = binary_acc(y_test_preds, y_tests)
    geometry = geometry_loss(fbeta, gbeta)
    geometry2 = geometry_loss(fbeta2, gbeta2)
    score = compute_score(np.round(y_train_preds.data.numpy()), np.round(y_trains.data.numpy()), weights)
    score2 = compute_score(np.round(y_test_preds.data.numpy()), np.round(y_tests.data.numpy()), weights)
    output_str = 'S{}/{} {:.2f} min |\n Train Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}, score: {:.3f} |\n Valid Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}, score: {:.3f}\n '.format(
        i, epoch, (time.time()-st)/60,
        avg_loss_train, acc, fmeasure, fbeta, gbeta, geometry, score, 
        avg_loss_test, acc2, fmeasure2, fbeta2, gbeta2, geometry2, score2)
    print(output_str)

#             output_str = 'S{}/{} {:.2f} min Train Loss: {:.6f} Valid Loss: {:.6f}'.format(i, epoch, (time.time()-st)/60, avg_loss_train, avg_loss_test)
#             print(output_str)
#             with open(saved_dir+'loss11_{}.txt'.format(i), 'a') as f:
#                 print(output_str, file=f)

#             early_stopping(avg_loss_test, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break

#     output_string = 'AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure|Geomotry\n{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}'.format(auroc2,auprc2,acc2,fmeasure2,fbeta2,gbeta2,geometry2)
#     print(output_string)     
#     with open(saved_dir+'score'+ str(i)+ '_epoch' + str(epoch) + '.txt', 'w') as f:
#         f.write(output_string)

#     avg_losses_train = np.array(avg_losses_train)
#     avg_losses_test = np.array(avg_losses_test)

#     np.save(saved_dir + 'avg_losses_train' + str(i) + '_epoch' + str(epoch), avg_losses_train)
#     np.save(saved_dir + 'avg_losses_test' + str(i) + '_epoch' + str(epoch), avg_losses_test)


In [ ]:
print(output_str)